In [29]:
import sagemaker

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

In [33]:
!pygmentize code/TrainingTensorFlowImageClassifier.py

import tensorflow as tf
from tensorflow import keras

import numpy as np

from s3fs.core import S3FileSystem
s3 = S3FileSystem()

import os

########
import smdistributed.dataparallel.tensorflow as sdp
sdp.init()
########

########
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
if gpus:
    tf.config.experimental.set_visible_devices(gpus[sdp.local_rank()], 'GPU')
########

bucket = 'zhixue.sagemaker.iad'
train_images = np.load(s3.open('{}/{}'.format(bucket, 'lego-simple-train-images.npy')))
train_labels = np.load(s3.open('{}/{}'.format(bucket, 'lego-simple-train-labels.npy')))
test_images = np.load(s3.open('{}/{}'.format(bucket, 'lego-simple-test-images.npy')))
test_labels = np.load(s3.open('{}/{}'.format(bucket, 'lego-simple-test-labels.npy')))

train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).repeat(200).shuffle(10000).batch(128)
test_ds = tf.data.Dataset.from_tensor_slices

In [34]:
from sagemaker.tensorflow import TensorFlow

estimator = TensorFlow(
                        base_job_name='DistributedTrainingTensorFlowImageClassifier',
                        source_dir='code',
                        entry_point='TrainingTensorFlowImageClassifier.py',
                        role=role,
                        py_version='py37',
                        framework_version='2.4.1',
                        # For training with multinode distributed training, set this count. Example: 2
                        instance_count=2,
                        # For training with p3dn instance use - ml.p3dn.24xlarge, with p4dn instance use - ml.p4d.24xlarge
                        instance_type= 'ml.p3.16xlarge',
                        sagemaker_session=sagemaker_session,
                        # Training using SMDataParallel Distributed Training Framework
                        distribution={'smdistributed':{
                                            'dataparallel':{
                                                    'enabled': True
                                             }
                                      }}
                        )

In [ ]:
estimator.fit()

2021-04-24 10:04:02 Starting - Starting the training job...
2021-04-24 10:04:25 Starting - Launching requested ML instancesProfilerReport-1619258641: InProgress
.........
2021-04-24 10:05:47 Starting - Preparing the instances for training.....

In [ ]:
model_data = estimator.model_data
print("Storing {} as model_data".format(model_data))
%store model_data